In [ ]:
import pandas as pd
import numpy as np
import pybullet as p
import time
import pybullet_data
import jsonpickle
import os
from numpy import pi

ASSET_ROOT = os.path.abspath('urdfc')
PLANE_ROOT = os.path.abspath('urdf')

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(PLANE_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

class PBObjectLoader:
    def __init__(self, asset_root):
        self.obj_ids = []
        self.obj_typ = []
        self.asset_root = os.path.abspath(asset_root)
        
    def load_obj(self, otype, pos=(0,0,0), ori=(0,0,0,1), wait=0):
        if isinstance(ori, list):
            ori = p.getQuaternionFromEuler(ori)
        
        oid = p.loadURDF(os.path.join(self.asset_root, f'{otype}.urdf'), pos, ori)

        for _ in range(wait):
            p.stepSimulation()
            time.sleep(1./240.)
            
        self.obj_ids.append(oid)
        self.obj_typ.append(otype)
            
        return oid
    
    def get_objs():
        return list(zip(self.obj_ids, self.obj_typ))


l = ['cube', 'cuboid', 'cylinder', 'pyramid', 'roof', 'cut_cuboid', 'thin_cuboid', 'small_cuboid']
loader = PBObjectLoader('urdfc')

loader.load_obj('cuboid', pos=[0.05, 0, 0.03], ori=[0, 0, 0])
loader.load_obj('cuboid', pos=[0, 0, 0.03], ori=[0, 0, 0])
# loader.load_obj('cube', pos=[0.025, 0, .05], ori=[pi/2, pi/2, 0], wait=100)

# loader.load_obj('cuboid', pos=[0.025, 0, .05], ori=[pi/2, pi/2, 0], wait=100)

o1, o2 = loader.obj_ids
# o1, = loader.obj_ids
prev = None
po = None

while True:
    cp = p.getClosestPoints(o2, o2, .05)[0]

    v1 = np.array(cp[5])
    v2 = np.array(cp[6])

    if np.linalg.norm(v1) < np.linalg.norm(v2):
        v1, v2 = v2, v1

    mp = v1 - v2
    mp[-1] = 2

    if po:
        p.removeBody(po)
    po = loader.load_obj('point', pos=mp)
    cp1 = p.getClosestPoints(o1, po, 5)[0]
    cp2 = p.getClosestPoints(o2, po, 5)[0]

    p.removeAllUserDebugItems()
    prev = p.addUserDebugLine(cp1[5], cp2[5])
    
    p.stepSimulation()
    time.sleep(1./240.)
    
#     (*_, cpos, corn), = p.getCollisionShapeData(o1, -1)
#     (*_, spos, sorn, _, _), = p.getVisualShapeData(o1, -1)

#     print(p.getCollisionShapeData(o1, -1))
    
#     print(f'cpos = {cpos}, spos = {spos}')
#     print(f'corn = {corn}, sorn = {sorn}')
    
#     print()

p.disconnect()
# #0.8708585184367256 0.13537291463132384 0.2942509320637464 1.0

pybullet build time: Nov 30 2022 16:56:59
